In [1]:
import traceback
from opcua import Client, ua
import time
from concurrent.futures._base import Error as OpcUaAccessError
from collections.abc import Iterable
import json
import numpy as np


class OpcUaClientAsServer:
	title = 'perception/server'

	def _init_(self, cfg, logger, server):
		self.cfg = cfg
		self.logger = logger
		self.ip, self.port = cfg[self.title + '/opcua_ip'], cfg[self.title + '/opcua_port']
		self.connected = False
		self.client = Client("opc.tcp://" + self.ip + ":" + str(self.port) + "/", timeout=2.05)  # Create client
		self.str2cmd = {'is_alive': lambda: True}
		self.ActiveCommandResult = None
		self.path = None
		self.server = server

	def bind(self):
		'''
		connect as client to OPCUA server and get children of Request Manager node
		'''
		if self.connected:
			return
		self.partlyConnected = False
		self.client.connect()
		self.partlyConnected = True

		self.browse_recursive(self.client.get_root_node())

		# init vars
		#		isWin = self.cfg[self.title+'/opcua_os']=='win'
		#		path = ["0:Objects", "0:Server", "4:CODESYS Softmotion RTE CX x64" if isWin else "4:CODESYS Control for Linux SL",
		#                        "3:Resources", "4:Application", "3:Programs", "4:RequestManager"]

		root = self.client.get_root_node().get_child(self.path)
		self.vars = dict()
		for c in root.get_children():
			key = c.get_browse_name().to_string()[2:]
			self.vars[key] = c

		self.connected = True
		self.logger.warn(f'OPCUA Client as a Server is connected to {self.ip}:{self.port}')

	def browse_recursive(self, node):
		'''
		browse nodes tree until reaching Request Manager
		save path to node in self.path
		'''
		for childId in node.get_children():
			ch = self.client.get_node(childId)
			if ch.get_browse_name().to_string() == '4:RequestManager':
				self.path = []
				for node in ch.get_path():
					self.path.append(node.get_browse_name().to_string())
				self.path = self.path[1:]
				return
			if self.path is None and ch.get_node_class() == ua.NodeClass.Object:  #
				self.browse_recursive(ch)

	def handle_request(self):
		request_type = None
		try:
			self.ActiveCommandResult = None
			self.bind()
			if self.vars['Start'].get_value():
				request_type = 'Start'
				self.parse_command(bit_type='Start')  # in case we got a command which is not a logging request
			if self.vars['Start_log'].get_value():
				request_type = 'Start_log'
				self.parse_command(bit_type='Start_log')  # in case we got a log request

		except (OpcUaAccessError, ConnectionError):
			if self.connected:
				self.logger.warn('Waiting for OPCUA server ...')
				self.connected = False
			time.sleep(0.01)

		except Exception:
			if self.connected:
				self.report_error('Error while parsing command:\n' + str(traceback.format_exc()), request_type)
			elif self.partlyConnected:
				self.logger.err_once('Bad OPCUA server data structure!')
			time.sleep(0.01)

	def parse_command(self, bit_type):
		if bit_type == 'Start':  # acknowledge the command was received (log commands are printed as the requested log)
			cmd = self.vars['ActiveCommand'].get_value()
			self.server.pt.log(f'received the following request: {cmd}', external_log=False)
		else:
			cmd = 'log_append'
		
		if cmd not in self.str2cmd:
			return self.report_error('Error: unknown command', bit_type)

		commands = self.vars['Commands']
		args = commands.get_child(["4:" + cmd, "4:Args"]).get_children()
		args = [a.get_value() for a in args]
		self.logger.warn('recieved command:' + str(cmd) + str(args))
		self.ActiveCommandResult = commands.get_child(["4:" + cmd, "4:Result"])

		res = self.str2cmd[cmd](*args)  # run cmd

		if not isinstance(res, Iterable) or isinstance(res, np.ndarray):
			res = [res]
		elif isinstance(res, str):
			if res[:5] == 'Error':
				return self.report_error(res, bit_type)
			res = [res]
		elif isinstance(res, dict):
			res = [json.dumps(res)]

		resVars = self.ActiveCommandResult.get_children()
		if len(resVars) != len(res):
			return self.report_error('Error: Result length mismatch ' + cmd + '->' + str(res), bit_type)

		for r, v in zip(res, resVars):
			if isinstance(r, np.ndarray):
				r = r.tolist()
			v.set_value(r)
		self.vars['Failure'].set_value(False)
		self.vars['ErrorDescription'].set_value('')
		self.vars[bit_type].set_value(False)  # mark action as done according to its type

	def report_error(self, description, bit_type):
		self.logger.error(description)
		try:
			self.ActiveCommandResult.get_child('4:success').set_value(False)
		except Exception:
			pass

		try:
			self.vars['Failure'].set_value(True)
			self.vars['ErrorDescription'].set_value(description[:250])
		except Exception:
			pass

		try:
			if bit_type is not None:
				self.vars[bit_type].set_value(False)
		except Exception:
			pass

	def update_commands(self, str2cmd):
		self.str2cmd.update(str2cmd)
		self.set_primary_camera = self.str2cmd['set_primary_camera']